In [1]:
from helpers.datasplit import S21SplitByThirds
from helpers.pipeline_manager import S21Pipeline
from algos.algorithms import S21DecisionTreeClassifier, S21DecisionTreeRegressor, S21RandomForestClassifier, S21GradientBoostingClassifier

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.tree import DecisionTreeClassifier
import pandas as pd

import warnings

In [2]:
warnings.filterwarnings("ignore")

df = pd.read_csv("../datasets/data/training.csv")

splitter = S21SplitByThirds(df)
df_train, df_val, df_test = splitter.split()

TARGET = "IsBadBuy"
drop_cols = [TARGET, "PurchDate"]

X_train = df_train.drop(columns=drop_cols)
X_val = df_val.drop(columns=drop_cols)
X_test = df_test.drop(columns=drop_cols)

y_train = df_train[TARGET]
y_val = df_val[TARGET]
y_test = df_test[TARGET]

Xs = [X_train, X_val, X_test]
ys = [y_train, y_val, y_test]

In [3]:
models = [
    ("S21DecisionTreeClassifier", S21DecisionTreeClassifier(random_state=42)), # max depth = 7
    ("sklearn DecisionTreeClassifier", DecisionTreeClassifier(random_state=42, max_depth=7)),
    ("S21DecisionTreeRegressor", S21DecisionTreeRegressor(random_state=42)),
    ("S21RandomForestClassifier", S21RandomForestClassifier(random_state=42)),
    ("S21GradientBoostingClassifier", S21GradientBoostingClassifier(number_of_trees=50, max_depth=3, learning_rate=0.1, random_state=42)),
]

for name, model in models:
    pipeline = S21Pipeline(name, model, Xs, ys)
    gini = pipeline.build_evaluate(X_val, y_val)
    print(f"{name} Gini: {gini:.5f}")

S21DecisionTreeClassifier Gini: 0.42693
sklearn DecisionTreeClassifier Gini: 0.42140
S21DecisionTreeRegressor Gini: 0.42661
S21RandomForestClassifier Gini: 0.46839
S21GradientBoostingClassifier Gini: 0.47210


Моя имплементация показывает результаты чуть лучше модели из sklearn. Происходит это из-за использования mergesort в методе _find_best_split().

In [4]:
search_configs = {
    "LGBMClassifier": {
        "estimator_cls": LGBMClassifier,
        "base_params": {
            "objective": "binary",
            "random_state": 42,
            "n_jobs": -1,
            "subsample_freq": 1,
            "force_col_wise": True,
            "verbosity": -1,
        },
        "param_distributions": {
            "n_estimators": [250, 400, 550, 700],
            "learning_rate": [0.03, 0.05, 0.08],
            "num_leaves": [31, 63, 95],
            "max_depth": [-1, 10, 16],
            "min_child_samples": [20, 40, 60],
            "subsample": [0.7, 0.85, 1.0],
            "colsample_bytree": [0.6, 0.8, 1.0],
            "reg_lambda": [0.0, 1.0, 5.0],
        },
        "n_iter": 12,
    },
    "CatBoostClassifier": {
        "estimator_cls": CatBoostClassifier,
        "base_params": {
            "loss_function": "Logloss",
            "eval_metric": "AUC",
            "verbose": False,
            "allow_writing_files": False,
            "random_seed": 42,
        },
        "param_distributions": {
            "iterations": [300, 500, 700],
            "learning_rate": [0.03, 0.05, 0.07],
            "depth": [4, 6, 8],
            "l2_leaf_reg": [1.0, 3.0, 7.0, 11.0],
            "bagging_temperature": [0.0, 0.5, 1.0],
            "border_count": [64, 128, 254],
        },
        "n_iter": 12,
    },
    "XGBClassifier": {
        "estimator_cls": XGBClassifier,
        "base_params": {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "use_label_encoder": False,
            "random_state": 42,
            "n_jobs": -1,
            "tree_method": "hist",
            "verbosity": 0,
        },
        "param_distributions": {
            "n_estimators": [300, 450, 600],
            "learning_rate": [0.03, 0.05, 0.08],
            "max_depth": [4, 6, 8],
            "min_child_weight": [1, 3, 5],
            "subsample": [0.7, 0.85, 1.0],
            "colsample_bytree": [0.6, 0.8, 1.0],
            "gamma": [0.0, 0.1, 0.3],
            "reg_lambda": [1.0, 3.0, 5.0],
        },
        "n_iter": 12,
    },
}

best_pipelines = {}

for label, cfg in search_configs.items():
    estimator_cls = cfg["estimator_cls"]
    base_params = cfg["base_params"]
    param_distributions = cfg["param_distributions"]
    n_iter = cfg.get("n_iter", 12)

    base_model = estimator_cls(**base_params)
    pipeline = S21Pipeline(label, base_model, Xs, ys)

    best_gini = pipeline.tune_model(
        estimator_cls,
        base_params=base_params,
        param_distributions=param_distributions,
        n_iter=n_iter,
        X_eval=X_val,
        y_eval=y_val,
    )
    
    best_pipelines[label] = pipeline

    print(f"Best {label} Gini: {best_gini:.5f}")
    print("")

Best LGBMClassifier Gini: 0.48859

Best CatBoostClassifier Gini: 0.49391

Best XGBClassifier Gini: 0.49297



- **LightGBM**: строит деревья листовым способом (leaf-wise), очень быстрый на больших и разреженных данных, но чувствителен к переобучению и требует аккуратной настройки регуляризации и `num_leaves`.
- **CatBoost**: использует симметричные деревья и ordered boosting, сразу работает устойчиво, нативно кодирует категориальные признаки и меньше зависит от тонкой настройки гиперпараметров.
- **XGBoost**: level-wise градиентный бустинг с большим набором регуляризаций, гибкими функциями потерь, но часто работает медленнее LightGBM и требует ручного кодирования категорий.

**CatBoost categorical feature**: библиотека сама преобразует категориальные столбцы через статистический (target-based) encoding с приёмом Ordered Target Statistics — для каждого объекта берётся информация только из «прошлых» примеров, добавляется сглаживание priors и генерируются комбинации категорий. Это снижает утечку target и позволяет обрабатывать категориальные признаки без One-Hot. (Conditional Target Rate).

**XGBoost DART mode**: режим Dropouts meet Additive Regression Trees. На каждой итерации случайно «выключается» часть уже обученных деревьев (dropout), модель обучается как если бы этих деревьев не было, а затем их веса подстраиваются. Такой стохастический приём напоминает dropout в нейросетях и помогает бороться с переобучением, за счёт параметров `sample_type`, `normalize_type`, `rate_drop`, `skip_drop` можно контролировать долю и способ выключения деревьев.



Лучший результат среди деревьев с бустингом у CatBoost (gini = 0.49391). Это произошло потому что библиотека хорошо подходит для решения таких задач.

In [5]:
catboost_pipe = best_pipelines["CatBoostClassifier"]
train_gini = catboost_pipe.build_evaluate(X_train, y_train)
val_gini = catboost_pipe.build_evaluate(X_val, y_val)
test_gini = catboost_pipe.build_evaluate(X_test, y_test)

print(f'train gini: {train_gini:.5f}, val gini: {val_gini:.5f}, test_gini: {test_gini:.5f}')

train gini: 0.55384, val gini: 0.49391, test_gini: 0.46724


Есть небольшое падение метрики, но, думаю некорректно считать это переобучением, так как качество метрики не резко падает, а плавно снижается. В целом, модель достаточно хорошо показывает себя на всех трех датасетах.

In [6]:
extra_clf_model = S21RandomForestClassifier(
    splitter='random', 
    bootstrap=False, # disabled subsampling
    random_state=42,
    max_depth=1 # on purpose
)

extra_clf_pipeline = S21Pipeline("S21ExtraTreesClassifiers", extra_clf_model, Xs, ys)
extra_gini = extra_clf_pipeline.build_evaluate()

print(f'S21ExtraTreesClassifier gini: {extra_gini:.5f}')

S21ExtraTreesClassifier gini: 0.20529


Результат модели на одиночном дереве > 0.12.